In [24]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

/Users/tom/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [13]:
nn_out = pd.read_csv('preds_for_logreg.csv')

In [15]:
nn_out.columns = ['nn_out', 'id']
nn_out.head()

,nn_out,id
0,0.024150,28170
1,0.117036,29836
2,0.030794,208192
3,0.319500,96826
4,0.562063,165584


In [9]:
train = pd.read_csv('train_with_sim_and_ents_long.csv')
train.head()

,Unnamed: 0,id,question1,question2,is_duplicate,entities1,entities2,similarity_score,CARDINAL_1,DATE_1,...,MONEY_2,NORP_2,ORDINAL_2,ORG_2,PERCENT_2,PERSON_2,PRODUCT_2,QUANTITY_2,TIME_2,WORK_OF_ART_2
0,0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,{u'GPE': 1},NaN,0.986684,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,{u'PERSON': 3},"{u'PERSON': 2, u'NORP': 1}",0.925000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
2,2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,{u'FAC': 1},{u'ORG': 1},0.929068,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,NaN,{u'CARDINAL': 1},0.730742,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,{u'CARDINAL': 1},NaN,0.846616,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
merged = train.merge(nn_out)

In [30]:
y = merged['is_duplicate']
x = merged[['similarity_score', 'nn_out']]
x.head()

AttributeError: 'Series' object has no attribute 'isna'

In [28]:

logit_model=sm.Logit(y,x)
result=logit_model.fit()
print(result.summary())


Optimization terminated successfully.
         Current function value: 0.432310
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:           is_duplicate   No. Observations:               306327
Model:                          Logit   Df Residuals:                   306325
Method:                           MLE   Df Model:                            1
Date:                Sun, 26 Nov 2017   Pseudo R-squ.:                  0.3437
Time:                        17:56:53   Log-Likelihood:            -1.3243e+05
converged:                       True   LL-Null:                   -2.0177e+05
                                        LLR p-value:                     0.000
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
similarity_score    -3.8013      0.013   -284.261      0.000      -3.828      -3.775
nn_out     

In [34]:
nn_out_test = pd.read_csv('test_preds_for_logreg.csv')
nn_out_test.columns = ['test_id','nn_out']
nn_out_test.head()

,test_id,nn_out
0,15,0.997517
1,20,0.813432
2,21,0.157851
3,23,0.006391
4,34,0.067587


In [35]:
test = pd.read_csv('test_with_sim_and_ents_long.csv')
test_merged = test.merge(nn_out_test)
test_merged.head()


,Unnamed: 0,test_id,question1,question2,entities1,entities2,similarity_score,CARDINAL_1,DATE_1,EVENT_1,...,NORP_2,ORDINAL_2,ORG_2,PERCENT_2,PERSON_2,PRODUCT_2,QUANTITY_2,TIME_2,WORK_OF_ART_2,nn_out
0,0,15,invalid,How will a Trump presidency affect the student...,NaN,"{u'ORG': 1, u'GPE': 2}",0.235347,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.997517
1,1,20,Why do rockets look white?,Why are rockets and boosters painted white?,NaN,NaN,0.892270,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.813432
2,2,21,What's causing someone to be jealous?,What can I do to avoid being jealous of someone?,NaN,NaN,0.956441,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.157851
3,3,23,How much is 30 kV in HP?,Where can I find a conversion chart for CC to ...,"{u'PRODUCT': 1, u'QUANTITY': 1}",{u'ORG': 1},0.835993,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006391
4,4,34,What is the best travel website in spain?,What is the best travel website?,{u'GPE': 1},NaN,0.968766,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.067587


In [38]:
x_test = test_merged[['similarity_score', 'nn_out']]

In [41]:
logreg = LogisticRegression()
logreg.fit(x, y)
y_pred = logreg.predict(x_test)

In [43]:
len(y_pred)

81126

In [48]:
predict = pd.DataFrame({"test_id":test_merged['test_id'], "pred_is_duplicate":y_pred})
predict.to_csv("logregattempt1.csv", index=False)

In [47]:
predict.head()

,pred_is_duplicate,test_id
0,1,15
1,1,20
2,0,21
3,0,23
4,0,34
